In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.matplotlib.rcParams['savefig.dpi'] = 144
import seaborn

# Recommendation Engine, Session 1

## Problem definition and data format

The goal of a recommendation engine is to match items to users that will prefer them.  Since we have a set of previous ratings, we can use them to make guesses about which items a user will rate highest.  In this tutorial, we will cover two ways of doing this:

1. **Content-based rating:** We can make this guess based only on features of the users or items.
1. **Collaborative Rating:** We could also look at ratings of similar users or similar items.

For an example data set, we are using the [MovieLens 10M dataset](http://files.grouplens.org/datasets/movielens/ml-10m.zip).  This contains about 10M ratings for 10k movies by 72k users.  We will be building applications that attempts to present movies that a given user would rate highly.  Extracting the zip file in the current directory should create a directory called `ml-10M100K/`.  The data wil be in several `.dat` files.  Let's take a look at the list of movies.

In [ ]:
with open('ml-10M100K/movies.dat', 'r') as f:
    for i in xrange(5):
        print f.readline()

Each line has three sections, separated by `::`.  It's easy enough to split them and read each line into a dictionary.
1. The first element is a key, which we will need to identify the movie in the other files.
1. Next is the title, with the year included for disambiguation.
1. We'll go ahead and read that out as a separate feature.
1. The last group is a list of categories, separated by `|`.  We'll convert this to a list.

The `parse_movie_line` function does that for a single line, returning a dictionary of key-value pairs.  Using list comprehension, we can easily run over the entire file.

In [ ]:
def parse_movie_line(l):
    id_, title, cats = l.strip().split('::')
    return {'id': int(id_), 'title': title, 'year': int(title.rsplit(' ')[-1][1:-1]), 
            'categories': cats.split('|')}

with open('ml-10M100K/movies.dat', 'r') as f:
    movies = [parse_movie_line(l) for l in f]

movies[:5]

This is using the built-in Python data types of lists and dictionaries. We'll get much better performance with the *Pandas* module.

If we feed Pandas our list of dictionaries, it does the right thing, making each dictionary a row, with the columns given by the dictionary keys.  By default, Pandas will assign a numerical index to the rows.  Since we have an id key for the movies, we'll use that as the index instead.

In [ ]:
import pandas as pd
df = pd.DataFrame(movies).set_index('id')
df.head()

We'll do the same thing with the tags and ratings data.

In [ ]:
def parse_tag_line(l):
    uid, mid, tag, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'tag': tag}

with open('ml-10M100K/tags.dat', 'r') as f:
    df_tags = pd.DataFrame([parse_tag_line(l) for l in f])

df_tags.head()

In [ ]:
def parse_rating_line(l):
    uid, mid, rating, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'rating': float(rating)}

with open('ml-10M100K/ratings.dat', 'r') as f:
    df_ratings = pd.DataFrame([parse_rating_line(l) for l in f])

df_ratings.head()

## Feature engineering

We will start with content modeling, based on the movie categories.  If a user rated a bunch of Adventure films highly, a simple scheme would be to recommend more Adventure films.  Of coure, many users may like several genres of movies, so we need a way to represent categories beyond just the string representing the favorite.

The first approach for vectorizing the categories might be to assign numbers to each of the categories.  For example, we could say *Adventure* = 1, *Drama* = 2, *Comedy* = 3, and so on.  This approach, however, signals that there is some order to these genres.  This mapping would suggest that *Drama* = (*Adventure* + *Comedy*)/2, which isn't meaningful.  We don't want a movie that is an *Adventure Comedy* to have the same feature as a *Drama*.

Instead, we need to recognize that genre is a **categorical variable**, and thus should be encoded with **one-hot encoding**.  Essentially, this give each genre its own dimension in feature space.  Our *Adventure Comedy* movie will be located along both the *Adventure* and *Comedy* axes, but at 0 along the *Drama* axis.

As before, we will use *Scikit Learn's* **transformers** to do this conversion.  We don't know the categories *a priori*, as we did with the days of the week, so we will use the `DictVectorizer` to help us.  This transformer takes in a list of dictionaries.  Each key in the dictionaries gets mapped to a column, and the values for those keys are placed in the appropriate column.  Columns for keys that are not present in a particular row are filled with zeros.

In order to use this, we need to transform our list of genres in to a dictionary, with values of one.

In [ ]:
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        def to_dict(l):
            try:
                return {x: 1 for x in l}
            except TypeError:
                return {}
        
        return X[self.col].apply(to_dict)

Once again, a pipeline helps us chain transformers together.  Since the pipeline doesn't end in an estimator, it acts as a transformer instead.

Transformers have a convenience `fit_transform()` method that simply calls `fit()` and then `transform()` immediately afterwards.

In [ ]:
cat_pipe = Pipeline([('encoder', DictEncoder('categories')),
                     ('vectorizer', DictVectorizer())])
features = cat_pipe.fit_transform(df)
features

The `DictVectorizer` returns a sparse matrix.  This allows efficient operations even on high-dimensional data.

## Nearest Neighbors

Now that we have a way to describe a movie, we want to be able to find other movies that are nearby in feature space.  This is the **nearest neighbors** problem, and Scikit Learn provides a class to handle this.

When the `NearestNeighbors` object is fit, it records all of the rows in such a way that it can efficiently look through them to find which is nearest to one queried later.  The `NearestNeighbors` assumes that the closest points are the ones for which the [Minkowski Distance](https://en.wikipedia.org/wiki/Minkowski_distance) is minimized.  That is, if we have two rows of our feature vector $X_{i\cdot}$ and $X_{j\cdot}$, it minimizes
$$ \|X_{i\cdot} - X_{j\cdot}\|_p^p = \sum_k \left(X_{ik} - X_{jk}\right)^p $$
By default, it computes $p=2$, which is the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance).

In [ ]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=20).fit(features)

We will discuss how to measure performance of these models later, but for now we will just eyeball the results.  We'll pick out two movies as test cases.  *Toy Story* happens to be the first movie in the data set.  *Dr. Strangelove* is a favorite of the author.

In [ ]:
df.iloc[[0, 737]]

The results for *Toy Story* are reasonable.  All of the movies are animated children's movies, and *Toy Story 2* shows up.  But by relying only on the genres, we are unable to select Pixar movies specifically, for example.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

This limitation is more apparent when looking a movies similar to *Dr. Strangelove*.  We get all the *War Comedy* movies, but that doesn't mean that they're similar to *Dr. Strangelove*.  While the author has never seen *The Wackiest Ship in the Army*, he is pretty sure it is of a rather different tone.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

## Tag Data

The problem is that we're only getting 20 bits of data about each movie from the categories.  We have much more information in the user-applied tags, though, so let's bring that to bear.

As a reminder, the tag data is stored one tag application per row.

In [ ]:
df_tags.head()

We want all tags for each movie, so we group by the movie_id and get a list of tags.

In [ ]:
all_tags = df_tags.groupby('movie_id')['tag'].apply(lambda x: x.tolist())
all_tags.head()

The equivalent of a SQL `join` statement in Pandas is the `merge()` method.  We specify a left join, to keep movies without any tags, and indicate rows should be matched by index, which is the movie_id.

In [ ]:
df = df.merge(all_tags.to_frame(), left_index=True, right_index=True, how='left')
df.head()

We'll use the same sort of pipeline to one-hot encode the tags.  (It might be better to use an alternative encoding that accounts for the number of times each tag was applied.)  Then, a `FeatureUnion` can join the two one-hot encoded matrices.

In [ ]:
tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer())])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [ ]:
features = union.fit_transform(df)
features

Over 16k features!

In [ ]:
nn = NearestNeighbors(n_neighbors=20).fit(features)
nn.fit(features)

With the tags, we now clearly pick out *Toy Story 2* as the most similar to *Toy Story*.  But the rest seem worse.  We're not getting any of the Pixar movies, for instance.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

The problem becomes more clear looking at *Dr. Strangelove*.  *The Mouse that Roared* is a good pick, but most of the rest have no tags.  The problem is that the tag space is so large, and so sparsely populated, that even similar movies will have very few overlapping tags.  As a result, movies with no tags end up being the closest to most movies.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

## Dimensional Reduction

We don't actually expect there to be 16k different concepts expressed in the tags.  Rather, several tags may be associated with a single underlying concept.  In the above, we can see tags for *animation* and *cartoon*, for example.

There are several methods of **dimensional reduction** that attempt to pull out a lower-dimensional structure.  We'll be using Truncated Singular Value Decomposition (SVD), which works well with sparse matrices.  SVD attempts to find orthogonal directions within feature space, along which the feature matrix has the most variation.  Recall that for $X$, an $n \times p$ feature matrix, we have the [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) with

$$ X = U \Sigma V^T $$

where $U$ is a unitary $n \times n$ matrix, $\Sigma$ is a diagonal $n \times p$ matrix, and $V$ is a unitary $p \times p$ matrix.  With this decomposition, we can then truncate the diagonal matrix $\Sigma$ to include only the $m$ dimensions with the most variation.

Our choice of $m=100$ seems reasonable, but can be tuned to adjust the sensitivity to the tags.

In [ ]:
from sklearn.decomposition import TruncatedSVD

tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer()),
                     ('svd', TruncatedSVD(n_components=100))])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [ ]:
features = union.fit_transform(df)
features

In [ ]:
nn = NearestNeighbors(n_neighbors=20).fit(features)
nn.fit(features)

Now we seem to be getting mostly Disney movies for *Toy Story*, which seems reasonable.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

Now, the algorithm finds movies like *The Great Dictator* and *MASH*, which are fairly similar in message to *Dr. Strangelove*.  It also finds *Full Metal Jacket*, another Stanley Kubrick war movie, albeit with a very different tone.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

We can examine the directions in feature space that the SVD picked out.  Here, we print the top ten tags associated with each dimension of the SVD.  We can see dimensions corresponding to classic movies, sci-fi franchises, comedies, and movies adapted from books.  (Note that the SVD has found that "based on a book", "adapted from:book", and "based on book" are all associated with each other.)

In [ ]:
svd = tag_pipe.named_steps['svd']
vect = tag_pipe.named_steps['vectorizer']
[", ".join([vect.feature_names_[i] for i in c.argsort()[:-10:-1]])
 for c in svd.components_]

## Recommendation for a User

The models so far seem to do a pretty good job of suggesting movies similar to a particular movie.  Sometimes, this is all that is needed.  If we notice a user buying a particular movie, we could use this to suggest similar movies they might like.

Often, however, we wish to suggest a movie to a user.  One approach would be to calculate an "average movie" that a particular user enjoyed, and then use the nearest neighbors approach to find movies similar to that one.

To help us judge, we'll add the movie titles to the ratings table.

In [ ]:
df_ratings_title = df_ratings.merge(df[['title']], left_on='movie_id', right_index=True)

And we'll look at user 9689, who seems to be a fan of horror.

In [ ]:
uid = 9689
df_ratings_title[df_ratings_title.user_id == uid].sort_values('rating', ascending=False)

We'll compute a weighted average movie, using the scores given as ratings.

In [ ]:
m_locs = df_ratings[df_ratings.user_id == uid]['movie_id'].apply(lambda x: df.index.get_loc(x))
scaled_ratings = df_ratings[df_ratings.user_id == uid]['rating'] * 0.2
weighted_avg_movie = scaled_ratings.reshape(1,-1).dot(features[m_locs,:].toarray()) / len(scaled_ratings)

And as expected, we get a bunch of horror movies.

In [ ]:
dists, indices = nn.kneighbors(weighted_avg_movie)
df.iloc[indices[0]]

## Cooperative Learning

Content-based recommendation systems work well to find items similar to those a user already likes.  To help with that, we turn to a collaborative learning model.  Instead of finding movies similar to movies a user liked, we will find *users* similar to a given user, and then find the movies that they rated highly.

How do we describe users?  We could use demographic information, but we have better set of features handy: the users' ratings.  Two users are similar if they have rated movies similarly.

In [ ]:
by_user_ratings = df_ratings.groupby('user_id').apply(
    lambda items: {i[1]: i[2] for i in items.itertuples()}) # 0 is index
features = DictVectorizer().fit_transform(by_user_ratings)

Previously, we have been using the default [Euclidean metric](https://en.wikipedia.org/wiki/Euclidean_distance) in the nearest neighbors calculation:

$$ d(x, y) =  \left| x - y \right|^2 \ . $$

For users, we will instead use the [cosine metric](https://en.wikipedia.org/wiki/Cosine_similarity),

$$ d(x, y) = 1 - \frac{ x\cdot y}{|x|\ |y|} \ , $$

which cares about angles between vectors in feature space.  This dependence on angle only lessens the effect of users using different scales to rate the movies

In [ ]:
nn = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute').fit(features)

In [ ]:
dists, indices = nn.kneighbors(features[by_user_ratings.index.get_loc(uid), :])
neighbors = [by_user_ratings.index[i] for i in indices[0]][1:]
ratings_grp = df_ratings_title[df_ratings_title['user_id'].isin(neighbors)] \
    .groupby('title')['rating']
len(ratings_grp)

## Average Rating

There are a bunch of movies to consider, but we want to pick those that were most enjoyed by the neighbor users.  We could pick those that have the highest average rating, but these are typically movies reviewed by only one or two users.

In [ ]:
ratings_grp.agg(['mean', 'count']).sort_values('mean', ascending=False)

Using the sum of the ratings will solve this particular problem, but will tend to highlight only those popular movies that almost everyone has seen.  There's no chance for a movie that's a hit with only some of the neighbors to shine through.

In [ ]:
ratings_grp.agg(['sum', 'count']).sort_values('sum', ascending=False)

One way to balance these two competing needs is to introduce a **Bayesian prior**.  This is the estimate we make that every movie has a rating $\mu$, before seeing any of the user ratings.  As we see ratings from users come it, we formulate a **posterior** estimate of the rating, taking into account both the prior and the new information provided by the ratings.  The posterior estimate of the rating, after seeing the $n$ reviews $\{x_i\}$, can be written as

$$ \frac{ \sum_{i=0}^n x_i + \mu N}{n + N} \ , $$

where $N$ reflects our level of confidence in the prior.  Adjusting $N$ affects the number of user reviews needed to push the posterior estimate away from $mu$.  Adjusting $mu$ affects where in the rankings movies with few reviews appear.  This technique is known as [Bayesian Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing).

In this case, we use $\mu = 3$, $N = 5$, essentially starting each movie off with five reviews of three stars.  The top rated movies are all ones reviewed by our user.  Looking a little bit down the list, we see a recommendation for *The Thing*, which seems very reasonable.  *Ghostbusters* also ranks highly.  This is somewhat surprising, since our user didn't rate any comedies.  Evidently, other fans of horror films like *Ghostbusters* anyway, so maybe our user should give it a try.

In [ ]:
def bayes_sum(N, mu):
    return lambda x: (x.sum() + mu*N) / (x.count() + N)

ratings_grp.aggregate(bayes_sum(5, 3)).sort_values(ascending=False)

## Exercises

1. Explore the effect of different distance metrics on the recommendations.  The 'euclidean', 'manhattan', and 'cosine' metrics are some of the more common ones.  Check the `NearestNeighbors` documentation for additional options.

2. Add in the year as a feature of movies.  Should it be a continuous or categorical variable?  While it's likely that a user who likes movies from 1970 will also like movies from 1971 (suggesting a continuous feature), a user liking movies from 1970 and 1990 is no guarantee that they like movies from 1980 (suggesting a categorical feature).  Can you find an encoding that's in between these two options?  How does the scale of the year, if treated as a continuous feature, affect the KNN calculation?  You may want to use a `StandardScaler`, from `sklearn.preprocessing`, to reduce this effect.

3. In weighting reviews, we consider not reviewing a movie to be less of a recommendation that scoring it a one.  Shift the rating scale to change this, and see how the affects the resultant recommendations.